#### This cleaning script loads data from the Social Security Adminstration and the Boston Earnings Report to create two output files for later analyses. Each section is commented to describe the process.

In [1]:
# Load requried libraries
import pandas as pd 
import requests, zipfile, io
import numpy as np

##### The SSA names database is stored as a series of text files in a zipped folder. The following code sends a request for the zip to the website and unzips the contents into 'z.' Then using list comprehension I return the name of all files that end with ".txt".

In [ ]:
r = requests.get("https://www.ssa.gov/oact/babynames/names.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
files = [info.filename for info in z.infolist() if info.filename[-4:] == ".txt"]

##### Next, I initialize an empty DataFrame with the same columns as the text files. Using a for loop, I iterate over alll the files in 'z' to open each document and read the information into a temporary DatFrame. The temp DataFrame is then appended to the final table.

In [ ]:
allNames = pd.DataFrame(columns=['name','gender','count'])
for i in range(len(files)):
    t = pd.read_csv(z.open(files[i]),
                    header = None,
                    names = ['name','gender','count'])
    allNames = allNames.append(t,ignore_index=True)

##### In order to account for cases where both men and women are assigned the same name, I sum all the counts of a name over time using a group by with name and gender. Then, after sorting in reverse order, I drop duplicate names. This means that if a name has more counts of men than women, I sort to put the male record above the female one. The drop duplicates will take the first instance of a record by default. 

In [ ]:
namesByCount  = allNames.groupby(['name','gender']).sum().reset_index()
uniqueNames = namesByCount.sort_values('count',ascending=False).drop_duplicates(['name'])
del(uniqueNames['count'])
uniqueNames.reset_index(drop='True',inplace=True)
uniqueNames['name'] = uniqueNames['name'].str.lower()
uniqueNames = uniqueNames.rename(columns={'name':'firstName'})

##### Below, I list all the csv files for the Boston Earnings Reports in a list to iterate over. 

In [ ]:
dataUrls = ["https://data.boston.gov/dataset/418983dc-7cae-42bb-88e4-d56f5adcf869/resource/70129b87-bd4e-49bb-aa09-77644da73503/download/employee-earnings-report-2017.csv",
           "https://data.boston.gov/dataset/418983dc-7cae-42bb-88e4-d56f5adcf869/resource/8368bd3d-3633-4927-8355-2a2f9811ab4f/download/employee-earnings-report-2016.csv",
           "https://data.boston.gov/dataset/418983dc-7cae-42bb-88e4-d56f5adcf869/resource/2ff6343f-850d-46e7-98d1-aca79b619fd6/download/employee-earnings-report-2015.csv",
           "https://data.boston.gov/dataset/418983dc-7cae-42bb-88e4-d56f5adcf869/resource/941c9de4-fb91-41bb-ad5a-43a35f5dc80f/download/employee-earnings-report-2014.csv",
           "https://data.boston.gov/dataset/418983dc-7cae-42bb-88e4-d56f5adcf869/resource/fac6a421-72fb-4f85-b4ac-4aca1e32d94e/download/employee-earnings-report-2013.csv",
           "https://data.boston.gov/dataset/418983dc-7cae-42bb-88e4-d56f5adcf869/resource/d96dd8ad-9396-484a-87af-4d15e9e2ccb2/download/employee-earnings-report-2012.csv",
           "https://data.boston.gov/dataset/418983dc-7cae-42bb-88e4-d56f5adcf869/resource/a861eff8-facc-4372-9b2d-262c2887b19e/download/employee-earnings-report-2011.csv"]

##### In this next set of code, I again create an empty shell for a DataFrame and iterate through the reports to append into the final output. In the second line of the for loop, I update the table names to be lower case and remove special characters. This allows me to standardize the column names across all years, as they don't match in the raw data. I also create a new column callled 'year' by extracting the year from the file name. This means that the final earnings table is long by employee and year.

In [ ]:
earnings = pd.DataFrame(columns=['name','department','title','total','year'])
for file in dataUrls:
    e = pd.read_csv(filepath_or_buffer= file,encoding='latin-1')
    e.columns = [n.lower().replace("_"," ").split(" ")[0] for n in list(e)]
    e = e[['name','department','title','total']]
    e['year'] = file[-8:-4]
    earnings = earnings.append(e,ignore_index=True)
earnings = earnings.rename(columns={'name':'fullName'})

##### With the extracted data, I move to cleaning up names. The names in the dataset are in the format Last,First Middle Initial. For the purpose of merging on Baby Names I only need the first name, so I use a string split on "," to get the last and FirstMiddle combinations into seperate columns. Then us

In [ ]:
earnings['fullName'] = earnings['fullName'].str.lower().replace('[^a-z, ]',"",regex = True)
earnings['last'] = earnings['fullName'].apply(lambda x: x.split(',')[0])
earnings['firstMiddle'] = earnings['fullName'].apply(lambda x: x.split(',')[1])
earnings['firstName'] = earnings['firstMiddle'].apply(lambda x: x.split(' ')[0])

##### Here I do a left merge of the unique names onto the earnings data using the newly created firstName column. 

In [ ]:
earnings = pd.merge(left = earnings,right = uniqueNames,on = 'firstName',how = 'left')

##### Then I check the number of records and unique users that do not have gender associates and drop them.

In [ ]:
print("Total Records missing gender information:",earnings.gender.isnull().sum(), "\n",
      "Unique Employees missing gender information:",earnings[earnings.gender.isnull()].fullName.nunique())

In [ ]:
earnings.dropna(inplace = True)

##### The first line of code below removes the non numeric characters from the total earnings column and converts the series to numeric. Then I subset the data to full time employees- meaning those employees who earned more than $29,348 in a year. This number is calculated using the living wage estimates as described in the attached report.

In [ ]:
earnings['total'] = pd.to_numeric(
    earnings['total'].str.strip().replace("[$,()]","",regex = True))
earnings = earnings[earnings['total'] >= 29348.00]

##### After some manual exploration of the data (not shown here) I found that in earlier years, schools at various levels were consolidated under the department title "Boston Public Schools" and in later years had specific school names. Using the regex pattern below I rename these locations as Boston Public Schools for consistency. I did not find similar issues with other department names in my manual exploration.

In [ ]:
earnings.department = np.where(earnings.department.str.contains(
    pat = "BPS|K-8|Middle|Elementary|Acad*|High|Sch*|EEC|EES|ELC|Svc|Pilot|Teachin",
    regex = True),
                               "Boston Public Schools",earnings.department)

##### The earnings dataset is still long by year and employee- for the later regression analysis I need the data unique to the individual level, with columns for total earnings in each year available. Using a pivot table I make a wide dataset that is unique to the name, department, title and gender of every employee. 

In [ ]:
wideEarnings = earnings.pivot_table(index = ['last','firstMiddle','department','title','gender'],
               columns = 'year', values = 'total').reset_index()

##### Rather than have new columns that are numeric years in string form, I change the column names to be explict about what they represent. 

In [ ]:
colNames = ["earnings" + x if x.startswith("2") else x for x in list(wideEarnings) ]
wideEarnings.columns = colNames

##### Finally I output both datasets for use in the following scripts.

In [ ]:
localPath = #insert your directory here
earnings.to_csv(path_or_buf=(localPath + "earningsLong.csv"), index = False)
wideEarnings.to_csv(path_or_buf=(localPath + "earningsByYear.csv"),index=False)